<a href="https://colab.research.google.com/github/KanishkGar/Eva/blob/main/S5/Session_2_Step_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Code 3
Target: 
- The previous model was underfitting so I'm going to change the structure of the model by adding GAP and more layers
- After fixing the model, I'll incorporate Data Transformations if necessary.
- Parameters don't need to be reduced because I got them under 10000 in the last iteration

Results: 
- Parameters: 9843
- Best Test Accuracy: 99.27%
- Best Training Accuracy: 98.0%

Analysis:
- I added a learning rate scheduler, changed the model, and added rotation to the train data for a better model. It had consistent accuracy towards the end and could be improved further with better knowledge of learning rates and tweaking the model.


In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
!/opt/bin/nvidia-smi
!nvcc --version

Sat Jan  2 23:13:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # input: 1x28x28
        # output: 6x14x14 
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 15, 3),
            # RF - 3x3
            nn.ReLU(),
            nn.BatchNorm2d(15),
            nn.Dropout(0.1),
            nn.Conv2d(15, 28, 3),
            # RF - 5x5
            nn.ReLU(),
            nn.BatchNorm2d(28),
            nn.Dropout(0.1),
            nn.Conv2d(28, 10, 1),
            nn.MaxPool2d(2, 2),
            # RF - 10x10
        )
        # input: 6x14x14
        # output: 12x7x7 
        self.conv2 = nn.Sequential(
            nn.Conv2d(10, 13, 3),
            # RF - 12x12
            nn.ReLU(),
            nn.BatchNorm2d(13),
            nn.Dropout(0.1),
            nn.Conv2d(13, 15, 3),
            # RF - 14x14
            nn.ReLU(),
            nn.BatchNorm2d(15),
            nn.Dropout(0.1),
            nn.Conv2d(15, 15, 3),
            # RF - 14x14
            nn.ReLU(),
            nn.BatchNorm2d(15),
            nn.Dropout(0.1),
        )
        # input: 12x7x7
        # output: 24x5x5 
        self.conv3 = nn.Sequential(
            nn.Conv2d(5, 10, 3, padding=1),
            # RF - 30x30
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Conv2d(10, 20, 3, padding=1)
            # RF - 32x32
            # nn.ReLU(),
            # nn.BatchNorm2d(20)
            #nn.Conv2d(20, 20, 3)
        )

        self.output = nn.Sequential(
            nn.AvgPool2d(kernel_size=6),
            nn.Conv2d(15, 15, 1),
            nn.BatchNorm2d(15),
            nn.ReLU(),
            nn.Dropout(.1),
            nn.Conv2d(15,10,1)
        )
      
                
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        #x = self.conv3(x)
        x = self.output(x);
        
        x = x.view(x.size(0), -1)
        return F.log_softmax(x)

In [11]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 15, 26, 26]             150
              ReLU-2           [-1, 15, 26, 26]               0
       BatchNorm2d-3           [-1, 15, 26, 26]              30
           Dropout-4           [-1, 15, 26, 26]               0
            Conv2d-5           [-1, 28, 24, 24]           3,808
              ReLU-6           [-1, 28, 24, 24]               0
       BatchNorm2d-7           [-1, 28, 24, 24]              56
           Dropout-8           [-1, 28, 24, 24]               0
            Conv2d-9           [-1, 10, 24, 24]             290
        MaxPool2d-10           [-1, 10, 12, 12]               0
           Conv2d-11           [-1, 13, 10, 10]           1,183
             ReLU-12           [-1, 13, 10, 10]               0
      BatchNorm2d-13           [-1, 13, 10, 10]              26
          Dropout-14           [-1, 13,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:73: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [20]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-10.5, 10.5), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                        #add more transforms here later
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [21]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        #tqdm._instances.clear()



def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [23]:
import torch.optim.lr_scheduler 

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, )
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_loader), 15 )

for epoch in range(1, 16):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

EPOCH: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:73: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.17093072831630707 batch_id=468 Accuracy=86.16: 100%|██████████| 469/469 [00:19<00:00, 24.24it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0724, Accuracy: 9792/10000 (98%)

EPOCH: 2


loss=0.13329261541366577 batch_id=468 Accuracy=95.44: 100%|██████████| 469/469 [00:18<00:00, 25.03it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0214, Accuracy: 9927/10000 (99%)

EPOCH: 14


loss=0.03832044079899788 batch_id=468 Accuracy=97.96: 100%|██████████| 469/469 [00:18<00:00, 24.74it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0229, Accuracy: 9922/10000 (99%)

EPOCH: 15


loss=0.04331941902637482 batch_id=468 Accuracy=98.00: 100%|██████████| 469/469 [00:19<00:00, 24.32it/s]



Test set: Average loss: 0.0229, Accuracy: 9924/10000 (99%)

